In [1]:
import sklearn
import pandas as pd 
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from functools import partial
from pathlib import Path
import os

In [2]:
df_covid = pd.read_csv("../../data_processed/usa_top30_with_density_smoothed.csv")
df_covid['date'] = pd.to_datetime(df_covid['date'], infer_datetime_format=True)

In [3]:
df_cams = pd.read_csv("../../data/cams_data.csv")
df_cams['time'] = pd.to_datetime(df_cams['time'], infer_datetime_format=True)
df_cams['time'] = df_cams['time'].apply(lambda x : x.date())

In [4]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39109 entries, 0 to 39108
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   shapeID                         39109 non-null  int64         
 1   GEOID                           39109 non-null  int64         
 2   NAME                            39109 non-null  object        
 3   date                            39109 non-null  datetime64[ns]
 4   cumulative_confirmed            39109 non-null  float64       
 5   smoothed_cumul                  39109 non-null  float64       
 6   smoothed_d1                     39109 non-null  float64       
 7   smoothed_d2                     39109 non-null  float64       
 8   mobility_transit_stations       39109 non-null  float64       
 9   mobility_retail_and_recreation  39109 non-null  float64       
 10  mobility_grocery_and_pharmacy   39109 non-null  float64       
 11  mo

In [5]:
df_cams = df_cams.groupby(['time','shapeID']).agg([np.mean,np.min,np.max])
df_cams

x0                            x1            \
                        mean      amin      amax      mean      amin   
time       shapeID                                                     
2020-01-01 0        2.746214  1.318739  4.143455 -0.468615 -1.177848   
           1        1.748343  1.136316  2.343562 -1.344321 -2.492807   
           2        2.537780  0.904072  3.824677 -0.879046 -1.434459   
           3        2.232631  1.758709  2.959195 -0.887527 -2.061887   
           4        1.863745  1.278517  2.611369 -1.410770 -2.542341   
...                      ...       ...       ...       ...       ...   
2020-10-09 3228    -2.578400 -3.521142 -1.635649 -0.764019 -1.945059   
           3229     1.005270 -1.254299  2.344047 -0.073360 -1.027821   
           3230    -0.703218 -1.961407  0.308347  1.287013 -0.388428   
           3231    -1.562584 -4.393294  1.607997 -1.495783 -2.678844   
           3232    -0.778073 -3.746977  1.462828 -1.150497 -2.576713   

                                    x2                            x3  ...  \
                        amax      mean      amin      amax      mean  ...   
time       shapeID                                                    ...   
2020-01-01 0       -0.009998 -0.196347 -1.236057  0.513239 -0.230875  ...   
           1        0.090072  0.043200 -1.023753  0.963664 -0.100307  ...   
           2       -0.098486 -0.283907 -1.005673  0.250384 -0.232938  ...   
           3        0.471877 -0.039456 -0.979300  0.596698 -0.188379  ...   
           4        0.008774 -0.033701 -1.106488  0.708281 -0.133756  ...   
...                      ...       ...       ...       ...       ...  ...   
2020-10-09 3228     0.639290  0.338535 -0.778648  1.246326  0.284701  ...   
           3229     0.781522 -0.528851 -1.276024  0.369275 -0.556423  ...   
           3230     3.819120  1.720019 -0.771418  3.415005  0.104234  ...   
           3231    -0.499571 -0.311439 -1.377934  0.928628  0.256350  ...   
           3232    -0.266170 -0.018446 -1.898842  2.290233  0.501033  ...   

                          x7        x8                            x9  \
                        amax      mean      amin      amax      mean   
time       shapeID                                                     
2020-01-01 0        1.068301 -0.245813 -0.744365  0.098474  0.402408   
           1        0.772818 -0.598203 -0.794981 -0.316423  0.132206   
           2        0.784380 -0.396858 -0.731261 -0.191591  0.468917   
           3        0.666151 -0.457559 -0.683913 -0.187589  0.202946   
           4        0.800916 -0.647023 -0.843513 -0.368641  0.175213   
...                      ...       ...       ...       ...       ...   
2020-10-09 3228     2.130849 -1.888306 -2.382600 -1.423180  0.709347   
           3229     0.873948 -1.050392 -1.614056 -0.613897  0.607270   
           3230     0.397674 -2.009938 -2.353993 -1.631919  0.183213   
           3231     2.137391 -1.170794 -1.769246 -0.547896  1.366944   
           3232     1.769830 -1.018824 -1.887357 -0.487967  1.203578   

                                             x10                      
                        amin      amax      mean      amin      amax  
time       shapeID                                                    
2020-01-01 0        0.005759  0.672307  0.705195  0.001659  1.147249  
           1       -0.040984  0.445009  0.150523 -0.042816  0.351716  
           2        0.106676  0.976562  0.247497 -0.065500  0.449746  
           3        0.009591  0.487794  0.349443  0.041207  0.643072  
           4        0.015534  0.445459  0.073232 -0.174049  0.272420  
...                      ...       ...       ...       ...       ...  
2020-10-09 3228     0.209706  1.485765  1.216620  0.623109  1.549771  
           3229    -0.264772  1.910146  0.485292  0.285741  0.770643  
           3230    -0.378370  0.791482  0.635360  0.159731  1.067685  
           3231     0.268053  2.583831  1.285421  0.152326  1.963216  
      

In [6]:
df_cams.columns = pd.Index([f"{x}_{y}" for (x,y) in df_cams.columns])
df_cams = df_cams.reset_index()

df_cams

,time,shapeID,x0_mean,x0_amin,x0_amax,x1_mean,x1_amin,x1_amax,x2_mean,x2_amin,...,x7_amax,x8_mean,x8_amin,x8_amax,x9_mean,x9_amin,x9_amax,x10_mean,x10_amin,x10_amax
0,2020-01-01,0,2.746214,1.318739,4.143455,-0.468615,-1.177848,-0.009998,-0.196347,-1.236057,...,1.068301,-0.245813,-0.744365,0.098474,0.402408,0.005759,0.672307,0.705195,0.001659,1.147249
1,2020-01-01,1,1.748343,1.136316,2.343562,-1.344321,-2.492807,0.090072,0.043200,-1.023753,...,0.772818,-0.598203,-0.794981,-0.316423,0.132206,-0.040984,0.445009,0.150523,-0.042816,0.351716
2,2020-01-01,2,2.537780,0.904072,3.824677,-0.879046,-1.434459,-0.098486,-0.283907,-1.005673,...,0.784380,-0.396858,-0.731261,-0.191591,0.468917,0.106676,0.976562,0.247497,-0.065500,0.449746
3,2020-01-01,3,2.232631,1.758709,2.959195,-0.887527,-2.061887,0.471877,-0.039456,-0.979300,...,0.666151,-0.457559,-0.683913,-0.187589,0.202946,0.009591,0.487794,0.349443,0.041207,0.643072
4,2020-01-01,4,1.863745,1.278517,2.611369,-1.410770,-2.542341,0.008774,-0.033701,-1.106488,...,0.800916,-0.647023,-0.843513,-0.368641,0.175213,0.015534,0.445459,0.073232,-0.174049,0.272420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914651,2020-10-09,3228,-2.578400,-3.521142,-1.635649,-0.764019,-1.945059,0.639290,0.338535,-0.778648,...,2.130849,-1.888306,-2.382600,-1.423180,0.709347,0.209706,1.485765,1.216620,0.623109,1.549771
914652,2020-10-09,3229,1.005270,-1.254299,2.344047,-0.073360,-1.027821,0.781522,-0.528851,-1.276024,...,0.873948,-1.050392,-1.614056,-0.613897,0.607270,-0.264772,1.910146,0.485292,0.285741,0.770643
914653,2020-10-09,3230,-0.703218,-1.961407,0.308347,1.287013,-0.388428,3.819120,1.720019,-0.771418,...,0.397674,-2.009938,-2.353993,-1.631919,0.183213,-0.378370,0.791482,0.635360,0.159731,1.067685
914654,2020-10-09,3231,-1.562584,-4.393294,1.607997,-1.495783,-2.678844,-0.499571,-0.311439,-1.377934,...,2.137391,-1.170794,-1.769246,-0.547896,1.366944,0.268053,2.583831,1.285421,0.152326,1.963216


In [7]:
df_cams['time'] = pd.to_datetime(df_cams['time'], infer_datetime_format=True)

In [8]:
df_total = pd.merge(df_cams, df_covid, how="inner", left_on=['time','shapeID'], right_on=['date','shapeID'])
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34130 entries, 0 to 34129
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   time                            34130 non-null  datetime64[ns]
 1   shapeID                         34130 non-null  int64         
 2   x0_mean                         34130 non-null  float64       
 3   x0_amin                         34130 non-null  float64       
 4   x0_amax                         34130 non-null  float64       
 5   x1_mean                         34130 non-null  float64       
 6   x1_amin                         34130 non-null  float64       
 7   x1_amax                         34130 non-null  float64       
 8   x2_mean                         34130 non-null  float64       
 9   x2_amin                         34130 non-null  float64       
 10  x2_amax                         34130 non-null  float64       
 11  x3

In [9]:
df_total.to_csv("../../data_processed/usa_top200_final.csv")